In [6]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 


#df = pd.read_csv("features/Features10too200hz.csv")
df = pd.read_csv("features/unfilteredFeatures.csv")

#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID != 'audible')]
df

,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.067106,0.065995,0.081097,224.738037,22.025275,0.003286,191.597839,224.738037,0.057210,682,...,0.051220,374,1713,6,106.306306,115.891113,1,1001,10010001,audible
1,0.063718,0.063090,0.075014,143.621399,12.683361,0.002283,103.957947,143.621399,0.046122,288,...,0.039666,128,1199,3,100.446429,114.028062,1,1001,10010002,audible
2,0.067169,0.066745,0.081422,272.503754,26.896239,0.003316,227.992035,272.503754,0.056197,816,...,0.057614,520,2043,15,112.871287,119.874289,1,1001,10010003,audible
3,0.065079,0.065057,0.078301,271.053162,25.535573,0.002825,224.132660,271.053162,0.053813,772,...,0.064233,596,2124,21,106.265060,117.951279,1,1001,10010004,audible
4,0.065251,0.065069,0.077894,306.745789,28.522983,0.002760,239.254150,306.745789,0.050894,756,...,0.058570,614,2374,18,116.417910,121.944233,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.105939,0.105803,0.114247,247.473846,30.490382,0.002267,61.923187,247.473846,0.026508,70,...,0.006785,0,1259,0,0.000000,90.523269,8,8019,80190246,whisper
4796,0.103561,0.103561,0.107324,197.284485,21.942521,0.000830,36.726807,197.284485,0.019279,10,...,0.005655,2,1043,0,0.000000,80.479425,8,8019,80190247,whisper
4797,0.103896,0.103946,0.108278,234.181610,26.425943,0.000987,47.587830,234.181610,0.021113,20,...,0.005838,2,1205,0,0.000000,81.305444,8,8019,80190248,whisper
4798,0.103268,0.103359,0.106641,176.794922,19.469253,0.000728,32.907990,176.794922,0.019222,6,...,0.005437,0,960,0,0.000000,70.628165,8,8019,80190249,whisper


In [7]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [8]:
def lda_user1():
    train_ids = [2,3,4,5,6,7,8]
    test_ids = [1]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
def lda_user2():
    train_ids = [1,3,4,5,6,7,8]
    test_ids = [2]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user3():
    train_ids = [1,2,4,5,6,7,8]
    test_ids = [3]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 
    
def lda_user4():
    train_ids = [1,2,3,5,6,7,8]
    test_ids = [4]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
        
def lda_user5():
    train_ids = [1,2,3,4,6,7,8]
    test_ids = [5]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score   
    
def lda_user6():
    train_ids = [1,2,3,4,5,7,8]
    test_ids = [6]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user7():
    train_ids = [1,2,3,4,5,6,8]
    test_ids = [7]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score    
    
def lda_user8():
    train_ids = [1,2,3,4,5,6,7]
    test_ids = [8]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = DecisionTreeClassifier()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 

In [9]:
def lda_users():
    a = lda_user1()
    b = lda_user2()
    c = lda_user3()
    d = lda_user4()
    e = lda_user5()
    f = lda_user6()
    g = lda_user7()
    h = lda_user8()
    scores = np.array([a,b,c,d,e,f,g,h])
    print("Durchschnitt: ")
    print (scores.mean())
    print(scores.std())

In [11]:
lda_users()

Accuracy for session 19 is 0.4177777777777778
Accuracy for session 19 is 0.31777777777777777
Accuracy for session 19 is 0.38666666666666666
Accuracy for session 19 is 0.49
Accuracy for session 19 is 0.4
Accuracy for session 19 is 0.56
Accuracy for session 19 is 0.35333333333333333
Accuracy for session 19 is 0.3476190476190476
Durchschnitt: 
0.40914682539682545
0.07517192595897058
